# Hydra Effect (Data)
Used to get various variables required for calculating Hydra effect over the dataset. 

## Setup

In [1]:
try:
    import google.colab
    IN_COLAB = True
    # %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    renderer = "colab"
except:
    IN_COLAB = False
    from IPython import get_ipython
    %load_ext autoreload
    %autoreload 2
    renderer = "jupyterlab"

In [2]:
%%bash
cd ../
pip install poetry
poetry install
cd notebooks

  Obtaining dependency information for platformdirs<4.0.0,>=3.0.0 from https://files.pythonhosted.org/packages/14/51/fe5a0d6ea589f0d4a1b97824fb518962ad48b27cd346dcdfa2405187997a/platformdirs-3.10.0-py3-none-any.whl.metadata
  Using cached platformdirs-3.10.0-py3-none-any.whl.metadata (11 kB)
Using cached platformdirs-3.10.0-py3-none-any.whl (17 kB)
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.8.0
    Uninstalling platformdirs-3.8.0:
      Successfully uninstalled platformdirs-3.8.0
Installing dependencies from lock file

Package operations: 0 installs, 1 update, 0 removals

  • Downgrading platformdirs (3.10.0 -> 3.8.0)

Installing the current project: transformer-lens (0.0.0)


In [3]:

# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = renderer

In [4]:
# Import stuff
import types
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader
import json
from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy
import subprocess

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
from tqdm import tqdm

In [5]:
import ast
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import torch
from matplotlib.cm import ScalarMappable
import pickle

In [6]:
if IN_COLAB: 
    import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens.utilities import devices
from transformer_lens.past_key_value_caching import HookedTransformerKeyValueCache
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [7]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [8]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def cuda():
    return torch.cuda.is_available()

def get_device(): 
    return "cuda" if cuda() else "cpu"

def save_pickle(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

# Function to load a pickle object from a file
def load_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

def save_json(data, file_path):
    with open(file_path, "w") as json_file:
        json.dump(data, json_file, indent=4)

def load_json(file_path):
    with open(file_path, "r") as json_file:
        loaded_data = json.load(json_file)
    return loaded_data

def get_gpu_usage(): 
    subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

device = get_device()
device

'cuda'

## Model

- fold_ln: Whether to fold in the LayerNorm weights to the subsequent linear layer. This does not change the computation.
- center_writing_weights: Whether to center weights writing to the residual stream (ie set mean to be zero). Due to LayerNormthis doesn't change the computation.
- center_unembed : Whether to center W_U (ie set mean to be zero). Softmax is translation invariant so this doesn't affect log probs or loss, but does change logits. Defaults to True.
- refactor_factored_attn_matrices: Whether to convert the factoredmatrices (W_Q & W_K, and W_O & W_V) to be "even". 

In [9]:
MODEL_NAME = "gpt2-large"
model = HookedTransformer.from_pretrained(
        MODEL_NAME,
        center_unembed=True,  
        center_writing_weights=True,              # Whether to center weights writing to the residual stream (ie set mean to be zero). Due to LayerNorm this doesn't change the computation.      
        fold_ln=True,                             # Whether to  fold in the LayerNorm weights to the subsequent linear layer.
        refactor_factored_attn_matrices=True,
    )


 

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


## Dataset
We use the CounterFact Dataset

In [10]:
dataset = load_json("data/fact_dataset.json")

In [11]:
def sample_dataset(dataset, idx = None): 
    if idx is None: 
        prompt = "The {} is located in"
        subject = "Eiffel Tower"
        target = "Paris"
    else: 
        sample = dataset[idx]
        prompt = sample["requested_rewrite"]["prompt"]
        subject = sample["requested_rewrite"]["subject"]
        target = sample["requested_rewrite"]["target_true"]["str"]
    return prompt, subject, target

In [12]:
sample = sample_dataset(dataset)
prompt, subject, target = sample
prompt = prompt.format(subject)

In [13]:
model.to_tokens(prompt).shape

torch.Size([1, 9])

## Ablation Methods: 
- Noise ablation 
- Resample ablation

In [14]:
def noise_ablation(prompt, subject, target, n_noise_samples=5, vx=3):
    subject_tokens = model.to_tokens(subject)
    
    #shape: batch, n_tokens, embedding_dim
    subject_embedding = model.embed(subject_tokens)
    _, n_tokens, embedding_dim = subject_embedding.shape
    
    #noise: N(0,v), v = 3*std(embedding)
    embedding = model.W_E
    v = vx*torch.std(embedding, dim=0) #for each v in V
    noise = torch.randn(
        (n_noise_samples, n_tokens, embedding_dim)
    ).to(device) + v
    
    subject_embedding_w_noise = subject_embedding + noise
    
    #shape: batch, n_tokens, vocab_size (logits)
    unembedded_subject = model.unembed(subject_embedding_w_noise)

    noisy_subject_tokens = torch.argmax(unembedded_subject, dim=-1)
    noisy_subject_str = [
        model.to_string(nst) for nst in noisy_subject_tokens
    ]
    true_prompt = prompt.format(subject)
    corrupted_prompts = [
        prompt.format(nss) for nss in noisy_subject_str
    ]
    return true_prompt, corrupted_prompts, target

In [15]:
prompt, subject, target = sample_dataset(dataset)
noise_ablation(prompt, subject, target)

('The Eiffel Tower is located in',
 ['The  volumeandomemedteenhai is located in',
  'The Legend HAallourers Salman is located in',
  'The ocl Duty apologoly Cursed is located in',
  'The  rim Ishatorsorea `` is located in',
  'The  Expandwaysroit mumLES is located in'],
 'Paris')

In [16]:
def resample_ablation(prompt, subject, target, n_noise_samples=20):
    subject_tokens = model.to_tokens(subject)
    embedding = model.W_E
    #we select n random rows from the embedding matrix
    permutations = torch.randperm(embedding.size(0))[:n_noise_samples]
    random_samples = embedding[permutations]
    #unsqueeze a token dimension between batch and embedding dims
    random_samples = random_samples.unsqueeze(dim=1)
    #we de-embed these rows
    random_embeddings = model.unembed(random_samples)
    random_tokens = torch.argmax(random_embeddings, dim=-1)
    random_subject_str = [
        model.to_string(t) for t in random_tokens
    ]
    corrupted_facts = [
        prompt.format(s[1:]) for s in random_subject_str
    ]
    true_fact = prompt.format(subject)
    return true_fact, corrupted_facts, target
    

    
    

In [17]:
resample_ablation(prompt, subject, target, n_noise_samples=5)

('The Eiffel Tower is located in',
 ['The ——————————————— is located in',
  'The Website is located in',
  'The Vita is located in',
  'The hould is located in',
  'The rocess is located in'],
 'Paris')

In [18]:
def pad_from_left(tokens : torch.tensor, maxlen:int):
    pad_token = model.tokenizer.pad_token_id
    padded_tokenized_inputs = torch.zeros(tokens.shape[0], maxlen)
    
    n_pads = maxlen - tokens.shape[-1]
    padded_tokenized_inputs[:,n_pads] = pad_token
    padded_tokenized_inputs[:,n_pads:] = tokens
    return padded_tokenized_inputs.long()

def pad_to_same_length(clean_tokens, corrupted_tokens): 
    
    maxlen = max([clean_tokens.shape[-1], corrupted_tokens.shape[-1]])
    
    if clean_tokens.shape[-1] > corrupted_tokens.shape[-1]: 
        corrupted_tokens = pad_from_left(corrupted_tokens, maxlen)
    elif clean_tokens.shape[-1] < corrupted_tokens.shape[-1]: 
        clean_tokens = pad_from_left(clean_tokens, maxlen)
    return clean_tokens, corrupted_tokens

In [19]:
import pickle

def save(tensor, filename):
    with open(filename, 'wb') as f:
        pickle.dump(tensor, f)

def load(filename):
    with open(filename, 'rb') as f:
        loaded_tensor = pickle.load(f)
    return loaded_tensor


In [20]:
def unembedding_function(residual_stack, cache, mlp=False) -> float:
    #we are only interested in applying the layer norm of the final layer on the final token
    #shape: [74, 5, 10, 1280] = n_layers, prompts, tokens, d_model
    z = cache.apply_ln_to_stack(residual_stack, layer = -1, mlp_input=mlp)
    z = z @ model.W_U
    return z


def delta_ablate(cache,
                  mle_token_idx,
                  target_token_idx, 
                return_labels=False):
    if return_labels: 
        residual_ablated_stack, labels = cache.decompose_resid(layer=-1, mlp_input=False, mode="all", return_labels=True)
    else: 
        residual_ablated_stack = cache.decompose_resid(layer=-1, mlp_input=False, mode="all", return_labels=False)
    
    #shape: [74, 4, 9, 50257] = n_layers, batch, tokens, vocab_size
    residual_logits = unembedding_function(residual_ablated_stack, cache)
    residual_logits = residual_logits[:,:,-1,:]

    target_idx_expanded = target_token_idx.repeat(residual_logits.shape[0],1,1)
    mle_idx_expanded = mle_token_idx.repeat(residual_logits.shape[0],1,1)

    target_residual_logits = residual_logits.gather(index=target_idx_expanded, dim=-1) - residual_logits.mean(dim=-1, keepdim=True)
    mle_residual_logits = residual_logits.gather(index=mle_idx_expanded, dim=-1) - residual_logits.mean(dim=-1, keepdim=True)
    
    if return_labels: 
        return labels, mle_residual_logits, target_residual_logits
    return mle_residual_logits, target_residual_logits

def patch_layer(
    corrupted_residual_component: Float[torch.Tensor, "batch pos d_model"],
    hook,
    cache):
    corrupted_residual_component[:, :, :] = cache[hook.name][:, :, :]
    """
    Restore a patch in clean run. 
    """
    return corrupted_residual_component

def extract_logit(logits, mle_token_idx, target_token_idx):     
    mle_logit = (logits.gather(dim=-1, index=mle_token_idx) - logits.mean(dim=-1, keepdim=True)).to("cpu")
    target_logit = (logits.gather(dim=-1, index=target_token_idx) - logits.mean(dim=-1, keepdim=True)).to("cpu")
    mle_logit = mle_logit.mean(dim=0)
    target_logit = target_logit.mean(dim=0)
    return mle_logit, target_logit

def run_all(clean_prompt: str,
                         corrupted_prompts: List[str],
                         target: str, 
                         corrupted_ablation=True):
    """
    activation_to_ablate: activation (resid_pre, attn_out, or mlp_out to ablate). resid_pre corresponds to the value of the residual
    stream at a layer and a position. attn_out and mlp_out correspond to the quantities added into the residual stream at a given 
    attention or mlp head. The values in the residual stream are preserved but the quantities being injected are corrupted. 
    
    mlp_input: Whether to include attn_out for the current
    layer - essentially decomposing the residual stream that's input to the MLP input rather than the Attn input.
    
    mode (str): Values are "all", "mlp" or "attn". "all" returns all
    components, "mlp" returns only the MLP components, and "attn" returns only the attention components.
    
    Here we get the compensatory effect of downstream layers following an ablation on the MLE token logits and the target token logits.
    """
    
    clean_tokens = model.to_tokens(clean_prompt, prepend_bos=True) 
    corrupted_tokens = model.to_tokens(corrupted_prompts, prepend_bos=True)
    #we take first token for the target. Not ideal though
    target_token_idx = model.to_tokens(target)[:,1] #remember to slice out the start pad token

    #we pad the clean and corrupted tokens to the same length (pad from left)
    clean_tokens, corrupted_tokens = pad_to_same_length(clean_tokens, corrupted_tokens)
    #repeat clean_tokens to match corrupted_tokens which have multiple noise samples. 
    clean_tokens = clean_tokens.expand(corrupted_tokens.shape[0], -1)
    target_token_idx = target_token_idx.expand(corrupted_tokens.shape[0], -1)
    
    #run the model on the clean and corrupted tokens, saving the model states for each
    #logits_shape: batch, n_tokens, vocab
    
    clean_logits, clean_cache = model.run_with_cache(clean_tokens, return_type="logits")
    corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens, return_type="logits")
    #The difference is states is attributed to the corrupted input. The difference in logit attributions
    #at each layer is the direct effect on that layer. 
    
    #if we want to inverse the corruption: set corrupted_ablation to False. This runs the corrupted prompts
    #and tracks the effect of restore clean states. For sake of clean code, we simply swap variables. 
    if not corrupted_ablation: 
        temp_cache = corrupted_cache
        temp_logits = corrupted_logits
        corrupted_cache = clean_cache
        corrupted_logits = clean_logits
        clean_cache = temp_cache
        clean_logits = temp_logits
    
    mle_token_idx = torch.argmax(clean_logits[:,-1,:], dim=-1).unsqueeze(-1)
    # layers, batch, 1 = direct effect at each layer due to the ablation
    
    clean_logits = clean_logits[:,-1,:]
    corrupted_logits = corrupted_logits[:,-1,:]
    
    clean_mle_logit, clean_target_logit = extract_logit(clean_logits, mle_token_idx, target_token_idx)
    corrupted_mle_logit, corrupted_target_logit = extract_logit(corrupted_logits, mle_token_idx, target_token_idx)
    
    
    layer_names, clean_mle_residual_logits, clean_target_residual_logits = delta_ablate(clean_cache,
                                                                          mle_token_idx,
                                                                          target_token_idx, 
                                                                         return_labels=True)
    
    corrupted_mle_residual_logits, corrupted_target_residual_logits = delta_ablate(corrupted_cache,
                                                                              mle_token_idx,
                                                                              target_token_idx)
    
    clean_mle_residual_logits = clean_mle_residual_logits.squeeze().mean(dim=-1)
    clean_target_residual_logits = clean_target_residual_logits.squeeze().mean(dim=-1)
    
    corrupted_mle_residual_logits = corrupted_mle_residual_logits.squeeze().mean(dim=-1)
    corrupted_target_residual_logits = corrupted_target_residual_logits.squeeze().mean(dim=-1)
        
    
    total_ablated_mle_residual_logits = torch.zeros(2*model.cfg.n_layers, len(layer_names), 
                                          device="cpu", dtype=torch.float32)
    total_ablated_target_residual_logits = torch.zeros(2*model.cfg.n_layers,len(layer_names),
                                             device="cpu", dtype=torch.float32)
    total_ablated_mle_logits = torch.zeros(2*model.cfg.n_layers,device="cpu", dtype=torch.float32)
    
    total_ablated_target_logits = torch.zeros(2*model.cfg.n_layers,device="cpu", dtype=torch.float32)

    for layer in range(2*model.cfg.n_layers):
        if layer%2==0: 
            activation_to_ablate = "attn_out"
        else: 
            activation_to_ablate = "mlp_out"
            
        #here we run the prompt, add a hook point from the corrupted_cache to ablate 
        #a position and layer. We then run with cache to get a saved state of the activations. 
        hook_fn = partial(patch_layer, cache=corrupted_cache)            
        with model.hooks(
            fwd_hooks = [(utils.get_act_name(activation_to_ablate, layer//2),hook_fn)]
        ) as hooked_model:
            restored_logits, ablated_cache = hooked_model.run_with_cache(clean_tokens, return_type="logits")
            # layers, batch, 1 = direct effect at each layer due to the ablation
            ablated_mle_residual_logits, ablated_target_residual_logits = delta_ablate(ablated_cache,
                                                                                          mle_token_idx,
                                                                                          target_token_idx, 
            )
            hooked_model.reset_hooks()
            model.reset_hooks()
            
        restored_logits = restored_logits[:,-1,:]
        
        mle_logit, target_logit = extract_logit(restored_logits, mle_token_idx, target_token_idx)
        
        total_ablated_mle_logits[layer] = mle_logit
        total_ablated_target_logits = target_logit
        
        total_ablated_mle_residual_logits[layer] = ablated_mle_residual_logits.squeeze().mean(dim=-1).to("cpu")
        total_ablated_target_residual_logits[layer] = ablated_target_residual_logits.squeeze().mean(dim=-1).to("cpu")


    return {
        "layer_names" : layer_names,
        "clean_mle_residual_logits" : clean_mle_residual_logits.to("cpu"),
        "clean_target_residual_logits" : clean_target_residual_logits.to("cpu"),
        "corrupted_mle_residual_logits" : corrupted_mle_residual_logits.to("cpu"),
        "corrupted_target_residual_logits" : corrupted_target_residual_logits.to("cpu"),
        "ablated_mle_residual_logits" : total_ablated_mle_residual_logits.to("cpu"),
        "ablated_target_residual_logits" : total_ablated_target_residual_logits.to("cpu"),
        "total_ablated_mle_logits" : total_ablated_mle_logits.to("cpu"),
        "total_ablated_target_logits" : total_ablated_target_logits.to("cpu"),
        
         "clean_mle_logit":clean_mle_logit,
         "clean_target_logit":clean_target_logit,
         "corrupted_mle_logit":corrupted_mle_logit,
         "corrupted_target_logit":corrupted_target_logit,
    
    }
    

In [21]:
def run_experiment(indices): 
    # indices = list(range(len(dataset)))
    # random.shuffle(indices)
    # indices = indices[:n]
    
    results = []
    
    for idx in indices: 
        prompt, subject, target = sample_dataset(dataset, idx=idx)
        true_fact, corrupted_facts, target = resample_ablation(prompt, subject, target, n_noise_samples=10)
        print(true_fact)
        result = run_all(
            clean_prompt=true_fact, 
            corrupted_prompts=corrupted_facts, 
            target=target, 
        )
        results.append(result)
    return results

# results_attn_CE = run_experiment_on_delta_ablate(dataset, n=2, activation_to_ablate="attn_out") 
# results_mlp = run_experiment_on_delta_ablate(dataset, n=10, activation_to_ablate="mlp_out") 
# results_resid = run_experiment_on_delta_ablate(dataset, n=10, activation_to_ablate="attn_out") 

In [ ]:
index_batches = [np.arange(i, 100+i) for i in [500, 600, 700, 800, 900, 1000]]


for indices in index_batches: 
    filename = f"hydra_results/{indices.min()}-{indices.max()}_results.pickle"
    results = run_experiment(indices)
    save(results, filename)
    print("saved ", filename)

# results_attn_CE = run_experiment_on_delta_ablate(dataset, n=2, activation_to_ablate="attn_out") 


Mario Vargas Llosa is a native speaker of
Google Patents, a product developed by
Pierre-Alexandre-Laurent Forfait is a native speaker of
Amazon Echo is developed by
The native language of Peter Kropotkin is
Intel 80286, a product developed by
Google Calendar is a product of
The original language of The Blue Planet is
Microsoft Office Live was developed by
Kensuke Tanabe has a citizenship from
Fernand Ledoux is a native speaker of
United Kingdom of Great Britain and Ireland belongs to the continent of
Kanna Arihara has a citizenship from
The original language of The Star Diaries is
Nonchan Noriben was created in the country of
Mataram Sultanate follows the religion of
The native language of David Beckham is
The official religion of Al-Mukhtar is
Google Image Labeler is owned by
Sukhoi, whose headquarters are in
The original language of Les Nanas was
The official religion of Bayazid Bastami is
SportsCentury premieres on
Microsoft Internet Explorer 6, a product created by
The mother tongu